In [1]:
#Q3
from tensorflow.keras.datasets import cifar10

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170508288/170498071 [==============================] - 156s 1us/step


In [3]:
import numpy as np

In [4]:
#data type
print((x_train.dtype, y_train.dtype, x_test.dtype, y_test.dtype))

(dtype('uint8'), dtype('uint8'), dtype('uint8'), dtype('uint8'))


In [5]:
#shape
print((x_train.shape, y_train.shape, x_test.shape, y_test.shape))

((50000, 32, 32, 3), (50000, 1), (10000, 32, 32, 3), (10000, 1))


In [6]:
from numpy.linalg import matrix_rank

In [7]:
#rank
print((matrix_rank(x_train),matrix_rank(y_train), matrix_rank(x_test), matrix_rank(y_test)))

(array([[3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 3, 3],
       ...,
       [3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 3, 3]]), 1, array([[3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 3, 3],
       ...,
       [3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 3, 3]]), 1)


In [9]:
#Q4
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [10]:
iris = load_iris()
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])

In [18]:
Cn = len(iris.target_names)
print ("{} class labels".format(Cn))

3 class labels


In [11]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [19]:
df.groupby(by=["target"]).count()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
target,,,,
0.0,50,50,50,50
1.0,50,50,50,50
2.0,50,50,50,50


In [20]:
prior = 1/3

In [47]:
np.random.seed(42)
msk = np.random.rand(len(df)) < 0.8
train = df[msk]
test = df[~msk]
len(test)

31

In [48]:
means = train.groupby(by=["target"]).mean()
means

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
target,,,,
0.0,5.009524,3.442857,1.464286,0.247619
1.0,5.927500,2.787500,4.257500,1.352500
2.0,6.629730,2.994595,5.586486,2.021622


In [49]:
var = train.groupby(by=["target"]).var()
var

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
target,,,,
0.0,0.139907,0.143972,0.033084,0.009384
1.0,0.263583,0.094968,0.241994,0.038968
2.0,0.439369,0.121637,0.310090,0.076186


,target
1,0.0


In [51]:
cp = np.zeros((len(test),Cn), dtype=float)

In [107]:
j = 0
for index, row in test.iterrows():
    p = np.array([0.0, 0.0, 0.0])
    for i in range(0,Cn):
        sl = np.multiply((1/(np.sqrt((2*np.pi*var.iloc[[i],[0]].to_numpy()[0][0])))), np.exp(-(np.power((row[0]-means.iloc[[i],[0]].to_numpy()[0][0]),2))/(2*var.iloc[[i],[0]].to_numpy()[0][0])))
        sw = np.multiply((1/(np.sqrt((2*np.pi*var.iloc[[i],[1]].to_numpy()[0][0])))), np.exp(-(np.power((row[1]-means.iloc[[i],[1]].to_numpy()[0][0]),2))/(2*var.iloc[[i],[1]].to_numpy()[0][0])))
        pl = np.multiply((1/(np.sqrt((2*np.pi*var.iloc[[i],[2]].to_numpy()[0][0])))), np.exp(-(np.power((row[2]-means.iloc[[i],[2]].to_numpy()[0][0]),2))/(2*var.iloc[[i],[2]].to_numpy()[0][0])))
        pw = np.multiply((1/(np.sqrt((2*np.pi*var.iloc[[i],[3]].to_numpy()[0][0])))), np.exp(-(np.power((row[3]-means.iloc[[i],[3]].to_numpy()[0][0]),2))/(2*var.iloc[[i],[3]].to_numpy()[0][0])))
        p[i] = prior * sl * sw * pl * pw
    
        
    cp[j] = p
    j += 1  

In [127]:
cp

array([[1.36280379e+000, 1.09246314e-016, 1.46492212e-024],
       [2.91539069e+000, 8.88706522e-017, 4.14788028e-024],
       [1.92389854e+000, 1.24757727e-016, 6.93750543e-024],
       [4.29686315e-001, 3.31662129e-018, 8.37512478e-026],
       [1.62496010e-001, 1.99008306e-020, 2.62953248e-026],
       [1.86935806e+000, 2.64370293e-016, 5.31330984e-024],
       [8.48078406e-001, 7.11396075e-018, 1.13735823e-025],
       [3.38266752e-003, 2.38730452e-012, 5.22558032e-020],
       [6.95218176e-106, 1.64081728e-002, 3.81183712e-003],
       [1.08152718e-119, 1.76166685e-002, 1.63843282e-002],
       [2.26765043e-070, 9.35815264e-002, 4.33573282e-006],
       [1.12556598e-087, 4.25493387e-001, 3.71731072e-004],
       [2.36934645e-059, 1.56517810e-002, 2.06732074e-007],
       [1.07309667e-060, 9.87291074e-002, 2.28326458e-006],
       [7.85039045e-058, 9.83322702e-002, 1.00568546e-006],
       [2.24863464e-092, 2.56949587e-001, 4.95695133e-004],
       [7.85118873e-055, 5.04689272e-002

In [125]:
y_pred = cp.argmax(axis=1)
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 1, 2, 2])

In [126]:
y_true = test["target"].to_numpy()
y_true

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])

In [121]:
from sklearn.metrics import accuracy_score

In [122]:
accuracy_score(y_true, y_pred)

0.967741935483871